Opencv to detect bubbles

In [1]:
import cv2

# Load the video
cap = cv2.VideoCapture('Data/plant2.mp4')

# Define the lower and upper bounds for the color of bubbles in HSV color space
lower_color = (0, 100, 100)
upper_color = (20, 255, 255)

# Loop through each frame of the video
while cap.isOpened():
    # Read the frame
    ret, frame = cap.read()
    if not ret:
        break
    
    # Convert the frame from BGR color space to HSV color space
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # Threshold the image to get a binary mask of the bubbles
    mask = cv2.inRange(hsv, lower_color, upper_color)
    
    # Find contours in the binary mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Loop through each contour
    for contour in contours:
        # Get the bounding rectangle of the contour
        x, y, w, h = cv2.boundingRect(contour)
        
        # Draw a rectangle around the bubble
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
    
    # Display the resulting frame
    cv2.imshow('Bubbles detection', frame)
    
    # Exit the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and close all windows
cap.release()
cv2.destroyAllWindows()


Opencv's hough circles transform method to detect bubbles

In [2]:
import cv2
import numpy as np

# Load the video
cap = cv2.VideoCapture('Data/plant2.mp4')

# Set the parameters for Hough Circles Transform
dp = 1
minDist = 50
param1 = 40
param2 = 15
minRadius = 5
maxRadius = 30

# Loop through each frame of the video
while cap.isOpened():
    # Read the frame
    ret, frame = cap.read()
    if not ret:
        break
    
    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Apply a median blur to reduce noise
    gray = cv2.medianBlur(gray, 5)
    
    # Detect circles using Hough Circles Transform
    circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, dp, minDist,
                               param1=param1, param2=param2,
                               minRadius=minRadius, maxRadius=maxRadius)
    
    # If any circles are found, draw a rectangle around them
    if circles is not None:
        circles = np.round(circles[0, :]).astype("int")
        for (x, y, r) in circles:
            cv2.rectangle(frame, (x-r, y-r), (x+r, y+r), (0, 255, 0), 2)
    
    # Display the resulting frame
    cv2.imshow('Bubbles detection', frame)
    
    # Exit the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and close all windows
cap.release()
cv2.destroyAllWindows()


subtract background first and then try to detect bubbles

In [4]:
import cv2

# Load the video and create a background subtractor object
cap = cv2.VideoCapture('plant2.mp4')
fgbg = cv2.createBackgroundSubtractorMOG2()

# Loop through each frame of the video
while cap.isOpened():
    # Read the frame
    ret, frame = cap.read()
    if not ret:
        break
    
    # Apply background subtraction to get the foreground mask
    fgmask = fgbg.apply(frame)
    
    # Apply thresholding to convert the mask to a binary image
    thresh = cv2.threshold(fgmask, 127, 255, cv2.THRESH_BINARY)[1]
    
    # Find contours in the binary image
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Loop through each contour
    for contour in contours:
        # Check if the contour has a circular shape and is large enough to be a bubble
        perimeter = cv2.arcLength(contour, True)
        area = cv2.contourArea(contour)
        circularity = 4 * np.pi * area / (perimeter ** 2)
        if circularity > 0.7 and area > 50:
            # Get the bounding rectangle of the contour
            x, y, w, h = cv2.boundingRect(contour)
            
            # Draw a rectangle around the bubble
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
    
    # Display the resulting frame
    cv2.imshow('Bubbles detection', frame)
    
    # Exit the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and close all windows
cap.release()
cv2.destroyAllWindows()


ZeroDivisionError: float division by zero

using blob detection to identify bubbles as round shiny circles

In [ ]:
import cv2
import numpy as np

# Load the video
cap = cv2.VideoCapture('Data/plant2.mp4')

# Loop through each frame of the video
while cap.isOpened():
    # Read the frame
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply a median blur to reduce noise
    gray = cv2.medianBlur(gray, 5)

    # Apply adaptive thresholding to obtain a binary image
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

    # Define the parameters for blob detection
    params = cv2.SimpleBlobDetector_Params()
    params.filterByCircularity = True
    params.minCircularity = 0.5
    params.filterByInertia = False
    params.filterByConvexity = False
    params.filterByColor = False
    params.filterByArea = True
    params.minArea = 100
    params.maxArea = 1000

    # Create the blob detector object and detect blobs
    detector = cv2.SimpleBlobDetector_create(params)
    keypoints = detector.detect(255 - thresh)

    # Loop through each blob and draw a rectangle around it
    for keypoint in keypoints:
        x, y = np.int16(keypoint.pt)
        r = np.int16(keypoint.size / 2)
        cv2.rectangle(frame, (x - r, y - r), (x + r, y + r), (0, 255, 0), 2)

    # Display the resulting frame
    cv2.imshow('Bubbles detection', frame)

    # Exit the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and close all windows
cap.release()
cv2.destroyAllWindows()


In [11]:



import cv2
import numpy as np

# Load the video
cap = cv2.VideoCapture('Data/plant1.mp4')

# Loop through each frame of the video
while cap.isOpened():
    # Read the frame
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply a median blur to reduce noise
    gray = cv2.medianBlur(gray, 5)

    # Apply adaptive thresholding to obtain a binary image
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

    # Define the parameters for blob detection
    params = cv2.SimpleBlobDetector_Params()
    
    # params.filterByColor = True
    # params.blobColor = 220

    # Filter by area
    params.filterByArea = True
    params.minArea = 50
    params.maxArea = 1000

    # # Filter by circularity
    params.filterByCircularity = True
    params.minCircularity = 0.2

    # Filter by convexity
    params.filterByConvexity = True
    params.minConvexity = 0.3   

    # Filter by inertia ratio
    params.filterByInertia = True
    params.minInertiaRatio = 0.2

    # Create the blob detector object and detect blobs
    detector = cv2.SimpleBlobDetector_create(params)
    keypoints = detector.detect(255 - thresh)

    # Loop through each blob and draw a rectangle around it
    for keypoint in keypoints:
        x, y = np.int16(keypoint.pt)
        r = np.int16(keypoint.size / 2)
        cv2.rectangle(frame, (x - r, y - r), (x + r, y + r), (0, 255, 0), 2)

    # Display the resulting frame
    cv2.imshow('Bubbles detection', frame)

    # Exit the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and close all windows
cap.release()
cv2.destroyAllWindows()



In [12]:
import cv2
import numpy as np
def get_bubbles(frame):
        if isinstance(frame, str):
            frame = cv2.imread(frame)
        # Convert the frame to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Apply a median blur to reduce noise
        gray = cv2.medianBlur(gray, 5)

        # Apply adaptive thresholding to obtain a binary image
        thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

        # Define the parameters for blob detection
        params = cv2.SimpleBlobDetector_Params()
        
        # params.filterByColor = True
        # params.blobColor = 220

        # Filter by area
        params.filterByArea = True
        params.minArea = 50
        params.maxArea = 1000

        # # Filter by circularity
        params.filterByCircularity = True
        params.minCircularity = 0.2

        # Filter by convexity
        params.filterByConvexity = True
        params.minConvexity = 0.3   

        # Filter by inertia ratio
        params.filterByInertia = True
        params.minInertiaRatio = 0.2

        # Create the blob detector object and detect blobs
        detector = cv2.SimpleBlobDetector_create(params)
        keypoints = detector.detect(255 - thresh)

        # Loop through each blob and draw a rectangle around it
        for keypoint in keypoints:
            x, y = np.int16(keypoint.pt)
            r = np.int16(keypoint.size / 2)
            cv2.rectangle(frame, (x - r, y - r), (x + r, y + r), (0, 255, 0), 2)
        
        return frame
    

        
 



113

: 